# Data Exploration and Data Prep

## Import Libraries

In [15]:
import pandas as pd
import numpy as np
import re
import csv

import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
np.random.seed(500)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/armaanbutt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/armaanbutt/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Data Load MIMIC-III
Load in `PRESCRIPTIONS` and `NOTEEVENTS` csvs into pandas dataframe.

In [4]:
df_prescriptions = pd.read_csv('../data/real-mimic-iii-database/PRESCRIPTIONS.csv', dtype=str)
df_noteevents = pd.read_csv('../data/real-mimic-iii-database/NOTEEVENTS.csv', dtype=str)

In [5]:
df_prescriptions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,00469061711,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,00056017275,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,00338055002,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,00054829725,20mg Tablet,20,mg,1,TAB,PO


In [6]:
df_noteevents.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


## Data Exploration

### Find most frequent prescriptions

In [7]:
df_most_frequent_prescriptions = df_prescriptions[['ROW_ID', 'NDC']][df_prescriptions['NDC'] != '0'].groupby('NDC').count().sort_values('ROW_ID',ascending=False)

In [8]:
# Use only top 5 for now.
df_most_frequent_prescriptions = df_most_frequent_prescriptions.head(20)
df_most_frequent_prescriptions = df_most_frequent_prescriptions.reset_index()
df_most_frequent_prescriptions

,NDC,ROW_ID
0,00338001702,67132
1,00338004904,60177
2,00338004903,57953
3,00409672924,55530
4,00517570425,53073
5,51079025520,51551
6,00338004902,51160
7,00338070341,47968
8,00338355248,44550
9,00338004938,44109


In [9]:
# Get Drug Names From Most Freq NDCs
df_most_frequent_prescriptions = df_most_frequent_prescriptions.rename(columns={'ROW_ID': 'FREQ'})
list_most_freq_prescription_ndcs = df_most_frequent_prescriptions['NDC'].to_list()
dict_most_freq_prescription_ndcs = df_prescriptions[df_prescriptions['NDC'].isin(list_most_freq_prescription_ndcs)][['NDC', 'DRUG']].groupby('DRUG').max().reset_index().to_dict(orient='records')

In [10]:
ndc_to_drug_names = {}

for record in dict_most_freq_prescription_ndcs:
    drug, ndc = record['DRUG'], record['NDC']
    if ndc not in ndc_to_drug_names:
        ndc_to_drug_names[ndc] = []

    # Remove non alpha numeric characters and make lowercase
    drug = re.sub('[^A-Za-z0-9]+', '', drug)
    drug = drug.lower()    
    ndc_to_drug_names[ndc].append(drug)

ndc_to_drug_names


{'00338004938': ['09sodiumchloride', '14ns', 'd5w', 'ns'],
 '00338001702': ['5dextrose', 'd5w01albumin', 'dextrose5'],
 '00074610204': ['enalaprilat', 'neoivfurosemide'],
 '00517570425': ['furosemide'],
 '00088222033': ['glargine',
  'insulin',
  'insulinglargine',
  'lantus',
  'lantus',
  'lantus',
  'insulin',
  'lantus'],
 '63323026201': ['heparin',
  'heparincrrtmachinepriming',
  'heparincrrt',
  'heparinflush1000unitsml',
  'heparinflush5000unitsml',
  'heparinflushcrrt5000unitsml',
  'heparinflushcvl100unitsml',
  'heparinsodium'],
 '58177020211': ['kdur',
  'potassiumchl',
  'potassiumchloride',
  'potassiumchloridepowder',
  'potassiumchloridereplacementoncology',
  'potas',
  'potass',
  'potassium',
  'potassiumch'],
 '58177000111': ['klor', 'microk', 'pota'],
 '00338011704': ['lr', 'lactatedringers'],
 '00008084199': ['lansoprazoleoralsuspension',
  'pantoprazole',
  'pantoprazolesodium',
  'panto'],
 '00409672924': ['magnesium', 'magnesium', 'magnesiumsulfatelatexfree'],


In [11]:
def clean_up_text(text):
    stopset = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    return [i for i in tokens if i not in stopset]

In [16]:
df_noteevents['TEXT_NORMALIZED'] = df_noteevents['TEXT'].apply(clean_up_text)
df_noteevents.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,TEXT_NORMALIZED
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,"[admission, date, :, [, *, *, 2151-7-16, *, *,..."
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...,"[admission, date, :, [, *, *, 2118-6-2, *, *, ..."
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...,"[admission, date, :, [, *, *, 2119-5-4, *, *, ..."
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...,"[admission, date, :, [, *, *, 2124-7-21, *, *,..."
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...,"[admission, date, :, [, *, *, 2162-3-3, *, *, ..."


In [17]:
df_noteevents.to_csv('../data/processed/NOTEEVENTS.csv', quoting=csv.QUOTE_ALL, quotechar='"')

In [93]:
# Save new dataframe with `Text_NORMALIZED` - Done
# Loop over NDC codes and find `TEXT_NORMALIZED` with drug names
# Save to dataframe with ROW_ID, TEXT_NORMALIZED, NDC

test_ndc = '00088222033'

drug_names = set(ndc_to_drug_names[test_ndc])

for index, row in df_noteevents.iterrows():
    print(row['TEXT_NORMALIZED'])

{'00338004938': ['09sodiumchloride', '14ns', 'd5w', 'ns'],
 '00338001702': ['5dextrose', 'd5w01albumin', 'dextrose5'],
 '00074610204': ['enalaprilat', 'neoivfurosemide'],
 '00517570425': ['furosemide'],
 '00088222033': ['glargine',
  'insulin',
  'insulinglargine',
  'lantus',
  'lantus',
  'lantus',
  'insulin',
  'lantus'],
 '63323026201': ['heparin',
  'heparincrrtmachinepriming',
  'heparincrrt',
  'heparinflush1000unitsml',
  'heparinflush5000unitsml',
  'heparinflushcrrt5000unitsml',
  'heparinflushcvl100unitsml',
  'heparinsodium'],
 '58177020211': ['kdur',
  'potassiumchl',
  'potassiumchloride',
  'potassiumchloridepowder',
  'potassiumchloridereplacementoncology',
  'potas',
  'potass',
  'potassium',
  'potassiumch'],
 '58177000111': ['klor', 'microk', 'pota'],
 '00338011704': ['lr', 'lactatedringers'],
 '00008084199': ['lansoprazoleoralsuspension',
  'pantoprazole',
  'pantoprazolesodium',
  'panto'],
 '00409672924': ['magnesium', 'magnesium', 'magnesiumsulfatelatexfree'],


Find all drug names for NDC `00338001702`

Discharge summary records have a list of medications `Medications on Admission` and `Medications on Transfer`. Might be better to take the 50 previous characters and 50 next characters of a text match of outside of sections.

In [ ]:
df_noteevents_with_drug = df_noteevents[df_noteevents['TEXT'].str.contains("5% Dextrose") == True]

In [ ]:
df_noteevents_with_drug

In [ ]:
df_noteevents_with_drug.iloc[0]['TEXT']